# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

#### Used cars are a big business and many factors go in to what makes one used car more or less valuable than other used cars.  In order to maximize profits a used car dealership will need to have an inventory of cars with desired features at a competitive price.  Therefore, it is imperative that the used car dealership has the appropriate data and projections to be able to create adequate inventory with the competitive prices.

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

The first step is to look at the dataset and see what can be gleaned.  The dataset being used contains detailed information about hundreds of thousands of used car sales. From this dataset we can begin to ask some important questions to help us better understand the data.
Questions to answer:
Are there features that have a strong correlation  or no correlation with price?
Unfortunately the data has few ‘continuous’ values to begin with. This will be addressed below.  With the only continuous values being id, price, year, and odometer reading very little was gleaned. With that said there appears to be little correlation with id and some correlation with odometer.
Are there features with exclusively one particular value?
Title_status is almost exclusively ‘clean’. Possibly drop this column since it likely can’t provide meaning.
Are there features with missing values?
Some columns have many missing values and these will need to be addressed on a case by case basis.  In some cases if there are not many missing values they might be dropped from the dataset.
Are there categorical features that need to be encoded?
This dataset has many categorical features; some of them will need to be one-hot encoded(ie Fuel), while other might be converted to ordinal values(ie Condition). Some of the categorical features with many different values might have to be addressed in a more unique way on a case by case basis.


### Data Preparation

After our initial exploration and fine tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

In [44]:
import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.linear_model import LogisticRegression, LinearRegression , Lasso, Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SequentialFeatureSelector, SelectFromModel, RFE
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
pd.set_option('display.max_columns', None)
from sklearn.inspection import permutation_importance
from numpy import arange
from pandas import read_csv
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold

In [45]:
cars = pd.read_csv('data/vehicles.csv')
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

In [46]:
cars.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
0,7222695916,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az
1,7218891961,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar
2,7221797935,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl
3,7222270760,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma
4,7210384030,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc


In [47]:
cars.describe()
cars.corr()

,id,price,year,odometer
id,1.000000,-0.002779,-0.059040,0.010721
price,-0.002779,1.000000,-0.004925,0.010032
year,-0.059040,-0.004925,1.000000,-0.157215
odometer,0.010721,0.010032,-0.157215,1.000000


In [48]:
n = cars.nunique(axis=0)
print("No.of.unique values in each column :\n",
      n)

No.of.unique values in each column :
 id              426880
region             404
price            15655
year               114
manufacturer        42
model            29649
condition            6
cylinders            8
fuel                 5
odometer        104870
title_status         6
transmission         3
VIN             118246
drive                3
size                 4
type                13
paint_color         12
state               51
dtype: int64


In [49]:
cars['paint_color'].unique()

array([nan, 'white', 'blue', 'red', 'black', 'silver', 'grey', 'brown',
       'yellow', 'orange', 'green', 'custom', 'purple'], dtype=object)

In [50]:
cars['region'].value_counts()
# cars.info()

columbus                   3608
jacksonville               3562
spokane / coeur d'alene    2988
eugene                     2985
fresno / madera            2983
                           ... 
meridian                     28
southwest MS                 14
kansas city                  11
fort smith, AR                9
west virginia (old)           8
Name: region, Length: 404, dtype: int64

In [51]:
# drop and fill missing values
cars.drop(columns = ['id', 'title_status', 'VIN', 'model', 'size', 'region'], inplace = True )
cars['manufacturer'].dropna(inplace = True)
cars['year'].fillna(cars['year'].median(), inplace = True)
#need to convert condition to ordinal and then fill missing with mean or median
# cylinders...convert to number and then fill with. mean or median
cars['fuel'].fillna(cars['fuel'].value_counts().index[0], inplace = True)
cars['odometer'].fillna(cars['odometer'].mean(), inplace = True)
cars['transmission'].fillna(cars['transmission'].value_counts().index[0], inplace = True)
# drive....what to do with missing...need to ohe 
# type....labels....missing?
cars['paint_color'].fillna(cars['paint_color'].value_counts().index[0], inplace = True)
cars = cars[cars['cylinders'] != 'other']
cars['cylinders'].fillna(0, inplace = True)
cars = cars[cars['drive'].notna()]
cars = cars[cars['manufacturer'].notna()]
cars = cars[cars['type'].notna()]

In [52]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 254712 entries, 31 to 426879
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         254712 non-null  int64  
 1   year          254712 non-null  float64
 2   manufacturer  254712 non-null  object 
 3   condition     166025 non-null  object 
 4   cylinders     254712 non-null  object 
 5   fuel          254712 non-null  object 
 6   odometer      254712 non-null  float64
 7   transmission  254712 non-null  object 
 8   drive         254712 non-null  object 
 9   type          254712 non-null  object 
 10  paint_color   254712 non-null  object 
 11  state         254712 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 25.3+ MB


In [55]:
## replace cylinders with numbers
cars['cylinders'] = cars['cylinders'].replace(['0'], 0)
cars['cylinders'] = cars['cylinders'].replace(['3 cylinders'], 3)
cars['cylinders'] = cars['cylinders'].replace(['4 cylinders'], 4)
cars['cylinders'] = cars['cylinders'].replace(['5 cylinders'], 5)
cars['cylinders'] = cars['cylinders'].replace(['6 cylinders'], 6)
cars['cylinders'] = cars['cylinders'].replace(['8 cylinders'], 8)
cars['cylinders'] = cars['cylinders'].replace(['10 cylinders'], 10)
cars['cylinders'] = cars['cylinders'].replace(['12 cylinders'], 12)
cars.loc[cars['cylinders'] == 0  , 'cylinders'] = 6
cars['cylinders'].unique()
cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 254712 entries, 31 to 426879
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         254712 non-null  int64  
 1   year          254712 non-null  float64
 2   manufacturer  254712 non-null  object 
 3   condition     166025 non-null  object 
 4   cylinders     254712 non-null  int64  
 5   fuel          254712 non-null  object 
 6   odometer      254712 non-null  float64
 7   transmission  254712 non-null  object 
 8   drive         254712 non-null  object 
 9   type          254712 non-null  object 
 10  paint_color   254712 non-null  object 
 11  state         254712 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 25.3+ MB


In [56]:
cars['fuel'].value_counts()

gas         217340
diesel       17088
other        15563
hybrid        3584
electric      1137
Name: fuel, dtype: int64

In [57]:
numeric_cols = ['price', 'year', 'odometer']
categorical_ohe = ['fuel', 'transmission', 'drive', 'paint_color', 'state']
categorical_ordinal = ['condition', 'cylinders', 'size',  ]
# Still to address: region, manufacturer, type, 

In [58]:
# Ordinal Encoding with Nan values and then replacing Nan values with -1
enc = OrdinalEncoder(categories = [['salvage', 'fair', 'good', 'excellent', 'like new', 'new']], dtype = object)
cars.loc[cars['condition'].notna(), ['condition']] = enc.fit_transform(cars['condition'].dropna().values.reshape(-1,1))
cars['condition'] = cars['condition'].fillna(-1)


In [59]:
cars2 = pd.get_dummies(cars,columns = ['fuel', 'transmission', 'drive'])
cars2 = cars2.reset_index()
cars2.info(verbose = True)
cars2.drop(columns = 'index', inplace= True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254712 entries, 0 to 254711
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   index                   254712 non-null  int64  
 1   price                   254712 non-null  int64  
 2   year                    254712 non-null  float64
 3   manufacturer            254712 non-null  object 
 4   condition               254712 non-null  int64  
 5   cylinders               254712 non-null  int64  
 6   odometer                254712 non-null  float64
 7   type                    254712 non-null  object 
 8   paint_color             254712 non-null  object 
 9   state                   254712 non-null  object 
 10  fuel_diesel             254712 non-null  uint8  
 11  fuel_electric           254712 non-null  uint8  
 12  fuel_gas                254712 non-null  uint8  
 13  fuel_hybrid             254712 non-null  uint8  
 14  fuel_other          

In [61]:
from sklearn.preprocessing import LabelEncoder
labeled_cols = ['manufacturer', 'paint_color', 'state', 'type']
cars2[labeled_cols] = cars2[labeled_cols].apply(LabelEncoder().fit_transform)
cars2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254712 entries, 0 to 254711
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   price                   254712 non-null  int64  
 1   year                    254712 non-null  float64
 2   manufacturer            254712 non-null  int64  
 3   condition               254712 non-null  int64  
 4   cylinders               254712 non-null  int64  
 5   odometer                254712 non-null  float64
 6   type                    254712 non-null  int64  
 7   paint_color             254712 non-null  int64  
 8   state                   254712 non-null  int64  
 9   fuel_diesel             254712 non-null  uint8  
 10  fuel_electric           254712 non-null  uint8  
 11  fuel_gas                254712 non-null  uint8  
 12  fuel_hybrid             254712 non-null  uint8  
 13  fuel_other              254712 non-null  uint8  
 14  transmission_automat

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.



In [62]:
features = cars2.drop(columns = ['price'])
# features = cars[['condition', 'odometer', 'cylinders', 'year']]
features.head()
price = cars2.price
price.head()

0    15000
1    27990
2    34590
3    35000
4    29990
Name: price, dtype: int64

In [63]:
#scale the data
object = StandardScaler()
features_scaled = pd.DataFrame(object.fit_transform(features), columns = features.columns)
print(type(features))
print(type(features_scaled))


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


## Linear Regression

In [68]:
f = LinearRegression(fit_intercept = True)
f.fit(features_scaled, price)
linreg_mse = mean_squared_error(f.predict(features_scaled), price)
print(linreg_mse)

59924409502268.41


In [67]:
cars2['prediction'] = f.predict(features_scaled)
# cars2.info(verbose = True)
cars2.head()

,price,year,manufacturer,condition,cylinders,odometer,type,paint_color,state,fuel_diesel,fuel_electric,fuel_gas,fuel_hybrid,fuel_other,transmission_automatic,transmission_manual,transmission_other,drive_4wd,drive_fwd,drive_rwd,prediction
0,15000,2013.0,13,3,6,128000.0,10,0,1,0,0,1,0,0,1,0,0,0,0,1,13825.896851
1,27990,2012.0,14,2,8,68696.0,8,0,1,0,0,1,0,0,0,0,1,1,0,0,49097.896851
2,34590,2016.0,7,2,6,29499.0,8,9,1,0,0,1,0,0,0,0,1,1,0,0,4297.896851
3,35000,2019.0,39,3,6,43000.0,10,5,1,0,0,1,0,0,1,0,0,1,0,0,104969.896851
4,29990,2016.0,7,2,6,17302.0,8,8,1,0,0,1,0,0,0,0,1,1,0,0,1481.896851


In [69]:
# checking the correlation between price and other features
pd.set_option('display.max_rows', 200)
price_corr = cars2.corrwith(cars2['price'])
print(price_corr)

price                     1.000000
year                     -0.001560
manufacturer              0.002727
condition                 0.002080
cylinders                 0.004392
odometer                  0.000706
type                      0.001477
paint_color               0.001643
state                     0.000930
fuel_diesel               0.001980
fuel_electric            -0.000125
fuel_gas                 -0.000961
fuel_hybrid              -0.000387
fuel_other               -0.000424
transmission_automatic    0.000957
transmission_manual      -0.000761
transmission_other       -0.000596
drive_4wd                 0.002135
drive_fwd                -0.002387
drive_rwd                 0.000223
prediction                0.006770
dtype: float64


In [70]:
# Create testing and training data
X = features_scaled
y = price
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=22)
print(type(X))

<class 'pandas.core.frame.DataFrame'>


In [110]:
#Simple Model
lr = LinearRegression().fit(X_train, y_train)
model_1_train_mse = mean_squared_error(y_train, lr.predict(X_train))
model_1_test_mse = mean_squared_error(y_test, lr.predict(X_test))
print(f'Train MSE: {model_1_train_mse: .2f}')
print(f'Test MSE: {model_1_test_mse: .2f}')

Train MSE:  85602821292534.09
Test MSE:  10472403325.27


In [112]:
print(list(zip(lr.coef_, X)))


[(-10908.951157314423, 'year'), (40465.0292065569, 'manufacturer'), (27123.271079435384, 'condition'), (49112.99704874912, 'cylinders'), (1816.355913702515, 'odometer'), (10542.977591863342, 'type'), (16931.90764855798, 'paint_color'), (9622.451842963905, 'state'), (-5.7573440373220104e+16, 'fuel_diesel'), (-1.5341395377160988e+16, 'fuel_electric'), (-8.142812126053694e+16, 'fuel_gas'), (-2.7105847006343316e+16, 'fuel_hybrid'), (-5.512039950143628e+16, 'fuel_other'), (1.867272941029412e+17, 'transmission_automatic'), (1.0700898073665429e+17, 'transmission_manual'), (1.6330153142111837e+17, 'transmission_other'), (1.8140758460410548e+16, 'drive_4wd'), (1.755319801483216e+16, 'drive_fwd'), (1.4298109022970638e+16, 'drive_rwd')]


In [121]:
coef_table = pd.DataFrame(list(X_train.columns)).copy()
coef_table.insert(len(coef_table.columns),"Coefs",lr.coef_.transpose())
coef_table.sort_values('Coefs')

,0,Coefs
10,fuel_gas,-8.142812e+16
8,fuel_diesel,-5.757344e+16
12,fuel_other,-5.512040e+16
11,fuel_hybrid,-2.710585e+16
9,fuel_electric,-1.534140e+16
0,year,-1.090895e+04
4,odometer,1.816356e+03
7,state,9.622452e+03
5,type,1.054298e+04
6,paint_color,1.693191e+04


In [72]:
r = permutation_importance(lr, X_test, y_test , n_repeats=30,random_state=0)
for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{X.columns[i]:<8}"
        f"  {r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")


transmission_automatic  16328632339519589058609152.000 +/- 49409728755061351251968.000
transmission_other  12452203915008386791374848.000 +/- 45104824689700466851840.000
transmission_manual  5389689359786892691767296.000 +/- 18805072586311167115264.000
fuel_gas  3161678864961046338076672.000 +/- 12581387601990568116224.000
fuel_diesel  1589623330471666406916096.000 +/- 5998945177698054438912.000
fuel_other  1445071397238038870360064.000 +/- 5398542157120941326336.000
fuel_hybrid  353158585274899852427264.000 +/- 1049335636674130083840.000
drive_4wd  155731815948781069795328.000 +/- 558453649341040427008.000
drive_fwd  145700070827319361536000.000 +/- 541700483256346279936.000
fuel_electric  107078338570130343067648.000 +/- 340089421488481239040.000
drive_rwd  96523207147630041759744.000 +/- 351711337340421013504.000
manufacturer  0.175 +/- 0.007
cylinders  0.147 +/- 0.009
condition  0.026 +/- 0.004


In [ ]:
### Do it all over again with code similar to this:
poly_ordinal_ohe = make_column_transformer((OrdinalEncoder(categories = [['Po', 'Fa', 'TA', 'Gd', 'Ex']]), ['HeatingQC']),
                                           (OneHotEncoder(drop = 'if_binary'), ['CentralAir']),
                                           (PolynomialFeatures(include_bias = False, degree = 2), ['OverallQual']))
pipe_3 = Pipeline([('transformer', poly_ordinal_ohe), 
                  ('linreg', LinearRegression())])
pipe_3.fit(X_train[['OverallQual', 'CentralAir', 'HeatingQC']], y_train)

## Ridge Regression


In [122]:
ridge = Ridge(alpha = 1)
ridge.fit(X_train, y_train)
ridge_train_mse = mean_squared_error(y_train, ridge.predict(X_train))
ridge_test_mse = mean_squared_error(y_test, ridge.predict(X_test))
print(f'The Ridge training MSE is: {ridge_train_mse}')
print(f'The Ridge testing MSE is: {ridge_test_mse}')
# scores = cross_val_score(ridge_pipeline, X_train, y_train, cv = 5, scoring='neg_root_mean_squared_error')
# print(scores)
# print(f'Mean score of {scores.mean()} with a standard deviation of {scores.std()}')

The Ridge training MSE is: 85602821060197.52
The Ridge testing MSE is: 10464144066.609045


In [123]:
# Using GridSearch to find best alpha
parameters = {'alpha': [0.1, .5, 1, 5, 10]}
gd = GridSearchCV(ridge, parameters)
gd.fit(X_train, y_train)
gd.best_params_

{'alpha': 10}

In [124]:
ridge2 = Ridge()
cv = RepeatedKFold(n_splits = 10, n_repeats = 3, random_state =1)
grid = dict()
grid['alpha'] = arange(0, 1, 0.01)
search = GridSearchCV(ridge2, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
results = search.fit(X_train, y_train)
print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

/Users/leostern/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=9.75237e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


MAE: -96095.683
Config: {'alpha': 0.0}


In [125]:
# With alpha 0 it seems that we are just using Linear Regression here.

In [126]:
r = permutation_importance(ridge, X_test, y_test , n_repeats=30,random_state=0)
for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{X.columns[i]:<8}"
        f"  {r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")

manufacturer  0.175 +/- 0.007
cylinders  0.144 +/- 0.009
condition  0.025 +/- 0.004
drive_rwd  0.017 +/- 0.001


In [127]:
# These appear to be the features that provide the most insight into price.

In [128]:
coef_table2 = pd.DataFrame(list(X_train.columns)).copy()
coef_table2.insert(len(coef_table2.columns),"Coefs",ridge.coef_.transpose())
coef_table.sort_values('Coefs')

,0,Coefs
10,fuel_gas,-8.142812e+16
8,fuel_diesel,-5.757344e+16
12,fuel_other,-5.512040e+16
11,fuel_hybrid,-2.710585e+16
9,fuel_electric,-1.534140e+16
0,year,-1.090895e+04
4,odometer,1.816356e+03
7,state,9.622452e+03
5,type,1.054298e+04
6,paint_color,1.693191e+04


## Feature Selection

In [ ]:
rfe = RFE(estimator = LinearRegression(), n_features_to_select = 3)
rfe.fit(X_train, y_train)
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(np.abs(X_train), price)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X_train.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features

In [ ]:
# Of these 10 features, surprisingly only manufacturer seems to have any crossover with the features found to be important using Ridge regression. 

In [85]:
## Feature Selection Pipeline
selector_pipe = Pipeline([
    ('selector', SequentialFeatureSelector(LinearRegression())),
    ('model', LinearRegression())
])
selector_pipe
selector_pipe.fit(X_train, y_train)
param_dict = {'n_features_to_select' : [3, 5, 10]}

Pipeline(steps=[('selector',
                 SequentialFeatureSelector(estimator=LinearRegression())),
                ('model', LinearRegression())])

In [86]:
param_dict = {'selector__n_features_to_select': [3,5,10]} #started with 2-5, adjusted...11 was best number of features
selector_grid = GridSearchCV(selector_pipe, param_grid=param_dict)
selector_grid.fit(X_train, y_train)
train_preds = selector_grid.predict(X_train)
test_preds = selector_grid.predict(X_test)
selector_train_mse = mean_squared_error(y_train, train_preds)
selector_test_mse = mean_squared_error(y_test, test_preds)
print(f'Train MSE: {selector_train_mse}')
print(f'Test MSE: {selector_test_mse}')
print(f'Best params: {selector_grid.best_params_}')

Train MSE: 85608307973636.58
Test MSE: 5150527910.820187
Best params: {'selector__n_features_to_select': 3}


In [87]:
r = permutation_importance(selector_pipe, X_test, y_test , n_repeats=30,random_state=0)
for i in r.importances_mean.argsort()[::-1]:
        print(f"{X.columns[i]:<8}"
        f"  {r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")

transmission_automatic  360142930112541773266944.000 +/- 1089776787175117422592.000
transmission_other  274644753526402857304064.000 +/- 994828188190484856832.000
transmission_manual  118874531440823180984320.000 +/- 414763086178613329920.000
fuel_gas  0.026 +/- 0.001
fuel_hybrid  0.011 +/- 0.001
drive_rwd  0.004 +/- 0.001
fuel_electric  0.001 +/- 0.000
fuel_other  0.000 +/- 0.000
drive_4wd  0.000 +/- 0.000
drive_fwd  0.000 +/- 0.000
fuel_diesel  0.000 +/- 0.000
paint_color  0.000 +/- 0.000
odometer  0.000 +/- 0.000
cylinders  0.000 +/- 0.000
condition  0.000 +/- 0.000
manufacturer  0.000 +/- 0.000
year      0.000 +/- 0.000
state     -0.000 +/- 0.001
type      -0.003 +/- 0.002


In [ ]:
coef_table3 = pd.DataFrame(list(X_train.columns)).copy()
coef_table3.insert(len(coef_table3.columns),"Coefs",selector_grid.coef_.transpose())
coef_table3.sort_values('Coefs')

### From this model it appears that transmission, fuel, and drive columns are most important


## Further Modeling
A few of the models seem to show that transmission, fuel, and drive are the most important features while another model showed odometer, state, and manufacturer were more important. Let's try running Linear Regression models in these 2 cases.



### Linear Regression with transmission, fuel, and drive

In [95]:
features3 = cars2[['transmission_automatic', 'transmission_other', 'transmission_manual', 'fuel_gas', 'fuel_hybrid', 'drive_rwd', 'fuel_electric','fuel_other', 'drive_4wd','drive_fwd','fuel_diesel']]      
price2 = cars2.price

In [96]:
# Create testing and training data
X1 = features3
y1 = price2
X1_train, X1_test, y1_train, y1_test = train_test_split(X1,y1,test_size=0.3,random_state=22)

In [109]:
tfd_model = LinearRegression().fit(X1_train, y1_train)
model_1_train_mse = mean_squared_error(y1_train, tfd_model.predict(X1_train))
model_1_test_mse = mean_squared_error(y1_test, tfd_model.predict(X1_test))
print(f'Train MSE: {model_1_train_mse: .2f}')
print(f'Test MSE: {model_1_test_mse: .2f}')

Train MSE:  85607314949744.03
Test MSE:  5898315861.89


### Linear Regression with Odometer, State, Manufacturer

In [99]:
features4 = cars2[['odometer', 'state', 'manufacturer']]
price3 = cars2.price

In [100]:
# Create testing and training data
X2 = features4
y2 = price3
X2_train, X2_test, y2_train, y2_test = train_test_split(X2,y2,test_size=0.3,random_state=22)

In [108]:
osm_model = LinearRegression().fit(X2_train, y2_train)
model_1_train_mse = mean_squared_error(y2_train, osm_model.predict(X2_train))
model_1_test_mse = mean_squared_error(y2_test, osm_model.predict(X2_test))
print(f'Train MSE: {model_1_train_mse: .2f}')
print(f'Test MSE: {model_1_test_mse: .2f}')

Train MSE:  85607219600213.25
Test MSE:  6298011663.58


### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

Modeling was a bit more challenging than expected.  Since a few of the columns lended themselves to one hot encoding it created a dataframe with far too many columns than was able to be processed with a home laptop.  With this restriction in mind a dataframe that was limited to 20 columns with several of the columns labeled in a less than ideal manner. The modeling produced inconsistent results as the errors were rather large and none of the features seemed to correlate strongly with the overall price.  While I could certainly recommend that automatic transmissions are recommended and preferred by buyers over other transmissions, I would not feel overly confident providing any other valuable insights with the data collected and analyzed thus far. I would recommend collecting more data that correlated more with price.  As we’ve seen from previous studies it might be possible to collect data on horsepower, fuel efficiency, or other metrics that might be  correlated better with price. Another fault of this whole process is that it’s unclear which types of cars provide the most profit vs. just which cars sell for the most.  This would be essential in making suggestions to used car dealers.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine tuning their inventory.

Upon collecting, organizing, and analyzing the data provided in order to give insight into how to maximize profits with the sale of used cars we can provide some insight but would also feel more comfortable collecting more data and providing further analysis.
But, with the data currently accessible a few things are clear:
Used car buyers prefer cars with automatic transmissions. Be very wary in using inventory with other types of transmissions.
Used car buyers are tempted with electric and hybrid vehicles thus it is recommended to have these types of vehicles available for purchase.
While other factors certainly contribute to the overall price and customer interest these variables vary wildly and it can not be fully ascertained at this time as to which factors would ultimately be influential in more profits.
